In [0]:
from mlflow import MlflowClient
import mlflow
from azureml.core import Experiment, Workspace

experiment_name = 'experiment_name'

In [0]:
ws = Workspace.get(name="workspace",
               subscription_id='subscription_id',
               resource_group='resource_group')

In [0]:
experiment = Experiment(ws, experiment_name)

In [0]:
a = []
for i in experiment.get_runs():
    a.append(i)
    
run = a[0]

In [0]:
model = run.register_model(model_name = 'housing_model_mlflow', model_path = 'LR_model/model.pkl')

### Depedency

In [0]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('housing_env')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==22.2.2'],
    pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib==0.17.0',
    'pandas==1.1.5',
    'scikit-learn==0.23.2'
])

### Deploying model

In [0]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment


script_file_name = "/dbfs/scripts/score.py"

inference_config = InferenceConfig(entry_script=script_file_name, environment=environment)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'Housing_data_service_mlflow')

aci_service_name = 'housingmlflowmodel-1'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig, overwrite=True)
aci_service.wait_for_deployment(True)
print(aci_service.state)

housingmlflowmodel-1
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-26 12:37:37+00:00 Creating Container Registry if not exists.
2022-08-26 12:37:38+00:00 Registering the environment.
2022-08-26 12:37:39+00:00 Use the existing image.
2022-08-26 12:37:39+00:00 Generating deployment configuration.
2022-08-26 12:37:39+00:00 Submitting deployment to compute.
2022-08-26 12:37:43+00:00 Checking the status of deployment housingmlflowmodel-1..
2022-08-26 12:39:35+00:00 Checking the status of inference endpoint housingmlflowmodel-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy